In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

from xgboost import plot_importance
from matplotlib import pyplot
from matplotlib.pyplot import figure

import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/train.csv")
pd.set_option('display.max_columns', None)
train_df.head()

In [ ]:
train_df.shape

In [ ]:
#train_df.corr()

In [ ]:
ax = sns.countplot(x="target", data=train_df)

In [ ]:
filter = train_df['target'] == 1
positive = train_df[filter]['target'].count()
filter = train_df['target'] == 0
negative = train_df[filter]['target'].count()
negative / positive

In [ ]:
y = train_df.pop('target')

In [ ]:
features_to_drop = ['id']

In [ ]:
X = train_df.drop(columns=features_to_drop)
#features = ['f21','f178','f68', 'f57', 'f42','f135','f213', 'f155','f77']
#X = X.loc[:, features]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 0, train_size=0.7, stratify=y)

In [ ]:
model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=1000, learning_rate=0.3,subsample=0.8,colsample_bytree=0.5)
model.fit(X_train, y_train, verbose=True, early_stopping_rounds=20, eval_metric='auc', eval_set=[(X_test, y_test)])

In [ ]:
print(model.feature_importances_)

In [ ]:
# plot feature importance
#columns = train_df.columns
importances = model.feature_importances_
#fig, ax = pyplot.subplots(figsize=(500, 500))
#plot_importance(model, ax=ax)
#pyplot.show()

In [ ]:
threshold = np.median(importances)
#threshold = np.percentile(importances, 90)
threshold

In [ ]:
selection = SelectFromModel(model, threshold=threshold, prefit=True)
select_X_train = selection.transform(X_train)
select_X_test = selection.transform(X_test)
model_selection = xgb.XGBClassifier(objective='binary:logistic', n_estimators=1000, learning_rate=0.05,subsample=0.8,colsample_bytree=0.2, min_child_weight=10, max_depth=3)
model_selection.fit(select_X_train, y_train, verbose=True, early_stopping_rounds=20, eval_metric='auc', eval_set=[(select_X_test, y_test)])

In [ ]:
plot_confusion_matrix(model_selection, select_X_test, y_test)

In [ ]:
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/test.csv")
test_df

In [ ]:
X_test = test_df.drop(columns=features_to_drop)
#X_test = X_test.loc[:, features]

In [ ]:
select_X_test = selection.transform(X_test)
y_pred = model_selection.predict(select_X_test)

In [ ]:
output = pd.DataFrame({'id': test_df.id, 'target': y_pred})
print(output.head(20))
output.to_csv('xgboost_submission.csv', index=False)